# Evolver Loop 4 Analysis: Comprehensive Source Exploration

## Key Insight from Evaluator
The evaluator correctly identified that we've only used 4 sources, but there are 15+ available pre-optimized CSVs. The Jonathan Chan kernel achieves better scores by ensembling from many more sources.

## Goal
1. Scan ALL available pre-optimized CSVs
2. Score each one
3. Build comprehensive ensemble picking best per-N
4. Identify which sources win for which N values

In [1]:
import pandas as pd
import numpy as np
from shapely.geometry import Polygon
import glob
import os
from tqdm import tqdm

# Tree geometry
TX = [0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075, -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125]
TY = [0.8, 0.5, 0.5, 0.25, 0.25, 0, 0, -0.2, -0.2, 0, 0, 0.25, 0.25, 0.5, 0.5]

def parse_value(s):
    if isinstance(s, str) and s.startswith('s'):
        return float(s[1:])
    return float(s)

def create_tree_polygon(x, y, deg):
    angle_rad = np.radians(deg)
    cos_a, sin_a = np.cos(angle_rad), np.sin(angle_rad)
    vertices = [(tx * cos_a - ty * sin_a + x, tx * sin_a + ty * cos_a + y) for tx, ty in zip(TX, TY)]
    return Polygon(vertices)

def compute_bounding_side(polygons):
    if not polygons:
        return 0
    all_points = []
    for poly in polygons:
        all_points.extend(list(poly.exterior.coords))
    all_points = np.array(all_points)
    return max(all_points.max(axis=0) - all_points.min(axis=0))

def compute_score_for_n(df, n):
    prefix = f"{n:03d}_"
    trees = df[df['id'].str.startswith(prefix)]
    if len(trees) != n:
        return float('inf')
    polygons = [create_tree_polygon(parse_value(row['x']), parse_value(row['y']), parse_value(row['deg'])) for _, row in trees.iterrows()]
    side = compute_bounding_side(polygons)
    return side**2 / n

def compute_total_score(df):
    return sum(compute_score_for_n(df, n) for n in range(1, 201))

print("Functions defined")

Functions defined


In [2]:
# Step 1: Find ALL CSV files in preoptimized directories

base_paths = [
    '/home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/',
    '/home/code/external_data/',
]

all_csvs = []
for base in base_paths:
    csvs = glob.glob(os.path.join(base, '**/*.csv'), recursive=True)
    all_csvs.extend(csvs)

print(f"Found {len(all_csvs)} CSV files:")
for csv in sorted(all_csvs):
    print(f"  {csv}")

Found 37 CSV files:
  /home/code/external_data/artemevstafyev/submission.csv
  /home/code/external_data/bucket/submission.csv
  /home/code/external_data/octaviograu/submission (80).csv
  /home/code/external_data/saspav/santa-2025.csv
  /home/code/external_data/saspav_latest/santa-2025.csv
  /home/code/external_data/smartmanoj_submission.csv
  /home/code/external_data/submission_best.csv
  /home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/best_ensemble.csv
  /home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/blended/submission (77).csv
  /home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/bucket-of-chump/submission.csv
  /home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/chistyakov/submission_best.csv
  /home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/ensemble.csv
  /home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/santa-2025-csv/santa-2025.csv
  /home/nonroot/snapshots/santa-2025/21116303805/code/pre

In [3]:
# Step 2: Score each CSV file

print("\nScoring all CSV files...")
scores = {}
valid_dfs = {}

for csv_path in tqdm(all_csvs):
    try:
        df = pd.read_csv(csv_path)
        # Check if it has the right columns
        if 'id' not in df.columns or 'x' not in df.columns or 'y' not in df.columns or 'deg' not in df.columns:
            print(f"  Skipping {csv_path} - missing columns")
            continue
        # Check if it has 200 configurations (N=1 to 200)
        n_configs = len(df['id'].str[:3].unique())
        if n_configs < 200:
            print(f"  Skipping {csv_path} - only {n_configs} configurations")
            continue
        
        score = compute_total_score(df)
        scores[csv_path] = score
        valid_dfs[csv_path] = df
        print(f"  {os.path.basename(csv_path)}: {score:.6f}")
    except Exception as e:
        print(f"  Error with {csv_path}: {e}")

print(f"\nSuccessfully scored {len(scores)} CSV files")


Scoring all CSV files...


  0%|          | 0/37 [00:00<?, ?it/s]

  3%|▎         | 1/37 [00:01<01:11,  1.98s/it]

  ensemble.csv: 70.676102


  5%|▌         | 2/37 [00:03<01:09,  1.99s/it]

...

 97%|█████████▋| 36/37 [01:10<00:01,  1.97s/it]

  submission (80).csv: 71.946272


100%|██████████| 37/37 [01:12<00:00,  1.97s/it]

100%|██████████| 37/37 [01:12<00:00,  1.97s/it]

  santa-2025.csv: 70.659959

Successfully scored 37 CSV files


In [4]:
# Step 3: Rank all sources by total score

print("\nRanking all sources by total score:")
print("="*80)
ranked = sorted(scores.items(), key=lambda x: x[1])
for i, (path, score) in enumerate(ranked[:20], 1):
    print(f"{i:2d}. {score:.6f} - {os.path.basename(path)}")

print(f"\nBest source: {os.path.basename(ranked[0][0])} with score {ranked[0][1]:.6f}")
print(f"Current baseline: 70.659959")
print(f"Target: 68.919154")


Ranking all sources by total score:
 1. 70.659959 - santa-2025.csv
 2. 70.659959 - santa-2025.csv
 3. 70.659982 - submission.csv
 4. 70.676102 - ensemble.csv
 5. 70.676102 - santa-2025.csv
 6. 70.676102 - best_ensemble.csv
 7. 70.676102 - santa-2025.csv
 8. 70.676501 - submission.csv
 9. 70.676501 - submission.csv
10. 70.743774 - smartmanoj_submission.csv
11. 70.773855 - submission.csv
12. 70.926150 - submission_70_926149550346.csv
13. 70.926150 - submission_best.csv
14. 70.926150 - submission_best.csv
15. 70.936674 - submission_70_936673758122.csv
16. 70.990692 - submission_opt1.csv
17. 71.946272 - submission (80).csv
18. 71.972027 - 71.97.csv
19. 71.972027 - 71.97.csv
20. 72.135010 - submission (77).csv

Best source: santa-2025.csv with score 70.659959
Current baseline: 70.659959
Target: 68.919154


In [5]:
# Step 4: Build comprehensive ensemble - pick best per-N from ALL sources

print("\nBuilding comprehensive ensemble (best per-N from all sources)...")

# For each N, find the best source
best_per_n = {}  # n -> (score, source_path)
for n in range(1, 201):
    best_score = float('inf')
    best_source = None
    for path, df in valid_dfs.items():
        score_n = compute_score_for_n(df, n)
        if score_n < best_score:
            best_score = score_n
            best_source = path
    best_per_n[n] = (best_score, best_source)

# Count wins per source
wins_per_source = {}
for n, (score, source) in best_per_n.items():
    source_name = os.path.basename(source)
    wins_per_source[source_name] = wins_per_source.get(source_name, 0) + 1

print("\nWins per source:")
for source, wins in sorted(wins_per_source.items(), key=lambda x: -x[1]):
    print(f"  {source}: {wins} N values")

# Calculate ensemble score
ensemble_score = sum(score for score, _ in best_per_n.values())
print(f"\nEnsemble score (best per-N from all sources): {ensemble_score:.6f}")
print(f"Best single source score: {ranked[0][1]:.6f}")
print(f"Improvement from ensemble: {ranked[0][1] - ensemble_score:.6f}")


Building comprehensive ensemble (best per-N from all sources)...



Wins per source:
  santa-2025.csv: 193 N values
  ensemble.csv: 5 N values
  submission.csv: 2 N values

Ensemble score (best per-N from all sources): 70.659959
Best single source score: 70.659959
Improvement from ensemble: 0.000000


In [6]:
# Step 5: Create the ensemble submission

print("\nCreating ensemble submission...")
ensemble_rows = []

for n in range(1, 201):
    score_n, source_path = best_per_n[n]
    df = valid_dfs[source_path]
    prefix = f"{n:03d}_"
    trees = df[df['id'].str.startswith(prefix)]
    for _, row in trees.iterrows():
        ensemble_rows.append(row.to_dict())

ensemble_df = pd.DataFrame(ensemble_rows)
print(f"Ensemble has {len(ensemble_df)} rows")

# Verify score
verify_score = compute_total_score(ensemble_df)
print(f"Verified ensemble score: {verify_score:.6f}")

# Save
ensemble_df.to_csv('/home/submission/submission.csv', index=False)
print("Saved to /home/submission/submission.csv")


Creating ensemble submission...


Ensemble has 20100 rows


Verified ensemble score: 70.659959
Saved to /home/submission/submission.csv


In [7]:
# Step 6: Analyze per-N improvements

print("\nPer-N analysis: Where did ensemble improve over best single source?")
print("="*80)

best_single_df = valid_dfs[ranked[0][0]]
improvements = []

for n in range(1, 201):
    single_score = compute_score_for_n(best_single_df, n)
    ensemble_score_n = best_per_n[n][0]
    diff = single_score - ensemble_score_n
    if diff > 1e-9:
        improvements.append((n, diff, single_score, ensemble_score_n, os.path.basename(best_per_n[n][1])))

print(f"\nEnsemble improved {len(improvements)} N values over best single source:")
for n, diff, single, ens, source in sorted(improvements, key=lambda x: -x[1])[:20]:
    print(f"  N={n:3d}: {single:.6f} -> {ens:.6f} (improved by {diff:.6f}) from {source}")


Per-N analysis: Where did ensemble improve over best single source?



Ensemble improved 0 N values over best single source:


In [8]:
# Step 7: Summary

print("="*80)
print("COMPREHENSIVE ENSEMBLE SUMMARY")
print("="*80)
print(f"\nTotal sources scanned: {len(all_csvs)}")
print(f"Valid sources: {len(valid_dfs)}")
print(f"\nBest single source: {os.path.basename(ranked[0][0])}")
print(f"Best single source score: {ranked[0][1]:.6f}")
print(f"\nEnsemble score: {verify_score:.6f}")
print(f"Improvement: {ranked[0][1] - verify_score:.6f}")
print(f"\nTarget: 68.919154")
print(f"Gap to target: {verify_score - 68.919154:.6f}")
print(f"\nSources contributing to ensemble:")
for source, wins in sorted(wins_per_source.items(), key=lambda x: -x[1]):
    print(f"  {source}: {wins} N values")

COMPREHENSIVE ENSEMBLE SUMMARY

Total sources scanned: 37
Valid sources: 37

Best single source: santa-2025.csv
Best single source score: 70.659959

Ensemble score: 70.659959
Improvement: 0.000000

Target: 68.919154
Gap to target: 1.740805

Sources contributing to ensemble:
  santa-2025.csv: 193 N values
  ensemble.csv: 5 N values
  submission.csv: 2 N values


In [9]:
# Let's look at which N values the other sources win
print("Detailed analysis of which sources win for which N values:")
print("="*80)

for n in range(1, 201):
    score_n, source_path = best_per_n[n]
    source_name = os.path.basename(source_path)
    if source_name != 'santa-2025.csv':
        # Get the best single source score for comparison
        best_single_score = compute_score_for_n(valid_dfs[ranked[0][0]], n)
        print(f"N={n:3d}: {source_name} wins with {score_n:.6f} (vs best single: {best_single_score:.6f}, diff: {best_single_score - score_n:.6f})")

Detailed analysis of which sources win for which N values:
N=  1: ensemble.csv wins with 0.661250 (vs best single: 0.661250, diff: 0.000000)
N=  2: submission.csv wins with 0.450779 (vs best single: 0.450779, diff: 0.000000)
N=  3: ensemble.csv wins with 0.434745 (vs best single: 0.434745, diff: 0.000000)
N=  4: submission.csv wins with 0.416545 (vs best single: 0.416545, diff: 0.000000)
N=  5: ensemble.csv wins with 0.416850 (vs best single: 0.416850, diff: 0.000000)
N= 17: ensemble.csv wins with 0.370040 (vs best single: 0.370040, diff: 0.000000)
N= 18: ensemble.csv wins with 0.368771 (vs best single: 0.368771, diff: 0.000000)
